![CoSAppLogo](images/cosapp.svg) **CoSApp** tutorials:

# Systems

## Simple Systems

A simple `System` does not contain subsystems.

### Import CoSApp core package

In [ ]:
# import cosapp base classes
from cosapp.systems import System
from cosapp.ports import Port

### Create a simple system

In [ ]:
class DemoPort(Port):
    
    def setup(self):
        self.add_variable('x')
        
        
class Multiply(System):

    def setup(self): # `setup` method defines the system structure
        self.add_input(DemoPort, 'p_in') # define a new input port
        self.add_inward('K1', 5.) # define a new inward variable
        self.add_outward('delta_x', 0.) # define a new outward variable
        self.add_output(DemoPort, 'p_out') # define a new output port

    def compute(self): # `compute` method defines what the system does
        self.p_out.x = self.p_in.x * self.K1
        self.delta_x = self.p_out.x - self.p_in.x

s = Multiply(name='mult')

![Multiple systems](images/systems_1.svg)

Run the system to confirm the expected behaviour

In [ ]:
s.p_in.x = 10.
s.K1 = 5.
s.run_once()

s.p_out

*More information about* `Ports` (`inputs` *and* `outputs`) *can be found in the* [Ports](02-Ports.ipynb) *tutorial*.

#### `inwards` and `outwards` variables

Every `System` has two specials kinds of variables:

- `inwards`: A inward data is an **input** variable needed by the system to compute its output. For example the pressure losses coefficient of a duct is a inwards of the duct to compute the output pressure from its inputs.
- `outwards`: A outward data is an **output** variable deduced from the inputs and useful as intermediate variable during the computation. For exemple the difference between the input and output pressure in a duct system is a local variable. Another example is the table object read from a filename (the filename being usually a *inwards*). 

All variables in **CoSApp** accept some additional informations:

- *unit*: Physical unit of the variable - given by a string. Units *are not enforced* inside a `System`.
This means that the user must ensure the computed variables in method `compute` are converted to the user
specified unit set in `setup`. **CoSApp** will take care of unit conversion between connected `System`s.
- *desc*: Short description of the variable.
- *dtype*: If you need to force certain data type(s) on a variable, a tuple of acceptable types can be provided
through this keyword. If that information is not supplied, dtype is inferred from the variable value; e.g.
a number (integer or floating point) will be typed as `Number`.

In [ ]:
class MultiplyAdvanced(System):

    def setup(self):
        self.add_input(DemoPort, 'p_in')
        # Inward and outward variables accept optional dtype and unit
        self.add_inward('K1', 5, dtype=int, desc='Multiplication coefficient')
        self.add_outward('delta_x', 0., unit='Pa', dtype=(int, float), 
                        desc='Spread between the output and the input')
        self.add_output(DemoPort, 'p_out')

    def compute(self):
        self.p_out.x = self.p_in.x * self.K1
        self.delta_x = self.p_out.x - self.p_in.x

advanced = MultiplyAdvanced(name='mult')

print('Inwards')
advanced.inwards

In [ ]:
print('Outwards')
advanced.outwards

## Complex Systems

A complex `System` may contain subsystems

### Create a complex system

Start with a new head system

In [ ]:
s = System('head')

Add some subsytems called `children` to your head system

In [ ]:
s.add_child(Multiply('mult1'))
s.add_child(Multiply('mult2'))

Plug systems (learn more about `Ports` in the [Ports](02-Ports.ipynb) tutorial)

In [ ]:
s.connect(s.mult2.p_in, s.mult1.p_out)

![Connection between Systems](images/systems_2.svg)

Run the system to confirm the expected behaviour

In [ ]:
s.mult1.p_in.x = 10.
s.mult1.K1 = 5.
s.mult2.K1 = 5.
s.run_once()

print('s.mult1.p_out')
s.mult1.p_out

In [ ]:
print('s.mult2.p_out')
s.mult2.p_out

Next you will learn more about [Ports](02-Ports.ipynb), the interface between `System`s.